In [143]:
import pandas as pd
import os

In [144]:
path = os.path.join("Resources", "purchase_data.csv")
df = pd.read_csv(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
Purchase ID    780 non-null int64
SN             780 non-null object
Age            780 non-null int64
Gender         780 non-null object
Item ID        780 non-null int64
Item Name      780 non-null object
Price          780 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.7+ KB


In [145]:
#Player Count
players = df['SN'].nunique()
print("There are %s unique players in this dataset." % players)

There are 576 unique players in this dataset.


In [146]:
# Purchasing Analysis (Total)

players = df['SN'].nunique()
items = df['Item ID'].nunique()
purchases = df['Purchase ID'].nunique()
revenue = df['Price'].sum()
avg_price = df['Price'].mean()

purchasingAnalysisTotal = pd.DataFrame({
    "Num of Items": [items],
    'Average Price': [avg_price],
    'Num of Purchases': [purchases],
    'Total Sales': [revenue],
})

purchasingAnalysisTotal['Average Price'] = purchasingAnalysisTotal['Average Price'].map('${:.2f}'.format)
purchasingAnalysisTotal['Total Sales'] = purchasingAnalysisTotal['Total Sales'].map('${:,.2f}'.format)
purchasingAnalysisTotal

,Num of Items,Average Price,Num of Purchases,Total Sales
0,183,$3.05,780,"$2,379.77"


In [147]:
#Gender Demographics

unique_playerDF = df.drop_duplicates('SN')
unique_playerDF['Gender'].value_counts()
numMale = (unique_playerDF['Gender'] == 'Male').sum()
numFemale = (unique_playerDF['Gender'] == 'Female').sum()
numOther = (unique_playerDF['Gender'] == 'Other / Non-Disclosed').sum()
genderDemographics = pd.DataFrame({
    'Total Count': [numMale, numFemale, numOther],
    'Percentage of Players': [(numMale/players)*100,(numFemale/players)*100, (numOther/players)*100],
}, ['Males', 'Females', 'Other / Non-Disclosed'])
genderDemographics['Percentage of Players'] = genderDemographics['Percentage of Players'].map('{:.2f}%'.format)
genderDemographics

,Total Count,Percentage of Players
Males,484,84.03%
Females,81,14.06%
Other / Non-Disclosed,11,1.91%


In [148]:
# Purchasing Analysis (Gender)
byGenderDF = df.groupby(['Gender'])
byGenderDF.sum(), byGenderDF.mean(), byGenderDF.count()

(                       Purchase ID    Age  Item ID    Price
 Gender                                                     
 Female                       42870   2412     9659   361.94
 Male                        255921  14942    60978  1967.64
 Other / Non-Disclosed         5019    363     1212    50.19,
                        Purchase ID        Age    Item ID     Price
 Gender                                                            
 Female                  379.380531  21.345133  85.477876  3.203009
 Male                    392.516871  22.917178  93.524540  3.017853
 Other / Non-Disclosed   334.600000  24.200000  80.800000  3.346000,
                        Purchase ID   SN  Age  Item ID  Item Name  Price
 Gender                                                                 
 Female                         113  113  113      113        113    113
 Male                           652  652  652      652        652    652
 Other / Non-Disclosed           15   15   15       15       

In [149]:
genderCount = byGenderDF['SN'].count()
genderMeanPrice = byGenderDF['Price'].mean()
genderTotal = byGenderDF['Price'].sum()
genderUnique = byGenderDF['SN'].nunique()
genderAvgPurch = (genderTotal/genderUnique)

purchasingAnalysisGender = pd.DataFrame({
    'Purchase Count': genderCount,
    'Average Purchase Price': genderMeanPrice.map('${:,.2f}'.format),
    'Total Purchase Sum': genderTotal.map('${:,.2f}'.format),
    'Avg Total Purchase per Person': genderAvgPurch.map('${:,.2f}'.format)},
    ['Female', 'Male', 'Other / Non-Disclosed'])
purchasingAnalysisGender.index.name = 'Gender'
purchasingAnalysisGender

,Purchase Count,Average Purchase Price,Total Purchase Sum,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [150]:
# Age Demographics
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
label_list = ["0-10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_groups = pd.cut(unique_playerDF['Age'], bins, labels = label_list)

ageDemographicsDF = pd.DataFrame({
    'Total Count': age_groups.value_counts().sort_index()
}, label_list)
ageDemographicsDF['Percentage of Players'] = ((ageDemographicsDF['Total Count']/players)*100).map('{:,.2f}%'.format)
ageDemographicsDF

,Total Count,Percentage of Players
0-10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [151]:
# Purchasing Analysis (Age)
df['Age Group'] = pd.cut(df['Age'], bins, labels = label_list)
byAgeGroupAll = df.groupby(['Age Group'])
ageUnique = byAgeGroupAll['SN'].nunique()
purchase_count = byAgeGroupAll['Purchase ID'].count()
avg_purchase_price = byAgeGroupAll['Price'].mean()
total_Value_by_age = byAgeGroupAll['Price'].sum()
avg_Total_purchase = (total_Value_by_age/ageUnique)

purchasingAnalysisAge = pd.DataFrame({
    'Purchase Count':purchase_count,
    'Average Purchase Price': avg_purchase_price.map('${:,.2f}'.format),
    'Total Purchase Value': total_Value_by_age.map('${:,.2f}'.format),
    'Avg Total Purchase per Person': avg_Total_purchase.map('${:,.2f}'.format),
}, label_list)
purchasingAnalysisAge.index.name = 'Age Groups'
purchasingAnalysisAge

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
0-10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [152]:
# Top Spenders
groupByName = df.groupby(['SN'])
individual_purchase_count = groupByName['SN'].count()
individual_purchase_total = groupByName['Price'].sum()
individual_purchase_average = groupByName['Price'].mean()

topSpendersDF = pd.DataFrame({
    'Purchase Count': individual_purchase_count,
    'Average Purchase Price': individual_purchase_average.map('${:,.2f}'.format),
    'Total Purchase Value': individual_purchase_total,
})
topSpendersDF = topSpendersDF.sort_values('Total Purchase Value', ascending=False)
topSpendersDF['Total Purchase Value'] = topSpendersDF['Total Purchase Value'].map('${:,.2f}'.format)
topSpendersDF.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [179]:
# Most Popular Items
itemDF = df[['Item ID', 'Item Name', 'Price']]
groupByItem = itemDF.groupby(['Item ID', 'Item Name'])
purchase_count = groupByItem.count().sort_values('Price', ascending=False)
